<h1>Coursera Capstone: Week 5 : Final Project</h1>
<h2>Welcome to my final project!</h2>
<h3>In this workbook we will:</h3>
<ul>
    <li>Download restaurant and hotel information from FourSquare for Raleigh, NC</li>
    <li>Performa a k-means analysis to find groupings of restaurants/hotels</li>
    <li>Overlay census income data on a choropleth map to identify area which may be favorable for a restaurant</h2>
</ul>

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
print('Geopy/Nominatim installed/imported')

!pip install folium
import folium # plotting library
print('Folium installed/imported')

from IPython.display import Image 
from IPython.core.display import HTML
from pandas.io.json import json_normalize

# libraries for IBM cloud access
import types
from botocore.client import Config
import ibm_boto3

print ('All libraries imported!')

Geopy/Nominatim installed/imported
     |████████████████████████████████| 92kB 15.3MB/s eta 0:00:01
Folium installed/imported
All libraries imported!


In [2]:
# The code was removed by Watson Studio for sharing.

<h3>Establish address for center of Raleigh, NC and set query/radius for FourSquare queries</h3>

In [3]:
address = '210 W Martin St., Raleigh, NC'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

restaurant_search_query = 'Restaurant'
hotel_search_query = 'Hotel'
radius = 4500
print(restaurant_search_query + ',' + hotel_search_query +' : OK!')
print(latitude, longitude)

Restaurant,Hotel : OK!
35.7770986824803 -78.6421750115139


In [4]:
# The code was removed by Watson Studio for sharing.

<h3>Verify that we obtained results from FourSquare OK</h3>

In [5]:
restaurant_results = requests.get(restaurant_url).json()
print ("Got restaurant results OK: {}".format(len(restaurant_results)))
hotel_results = requests.get(hotel_url).json()
print ("Got hotel results OK: {}".format(len(hotel_results)))


Got restaurant results OK: 2
Got hotel results OK: 2


<h3>Extract only venue data from responses</h3>

In [6]:
# assign relevant part of JSON to venues
restaurant_venues = restaurant_results['response']['venues']
hotel_venues = hotel_results['response']['venues']

# tranform venues into a dataframe
restaurant_dataframe = json_normalize(restaurant_venues)
print(restaurant_dataframe.head())

hotel_dataframe = json_normalize(hotel_venues)
print(hotel_dataframe.head())

                                          categories delivery.id  \
0  [{'id': '4bf58dd8d48988d1db931735', 'name': 'T...         NaN   
1  [{'id': '4bf58dd8d48988d18e941735', 'name': 'C...         NaN   
2  [{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...         NaN   
3  [{'id': '4bf58dd8d48988d14f941735', 'name': 'S...         NaN   
4  [{'id': '4bf58dd8d48988d142941735', 'name': 'A...         NaN   

  delivery.provider.icon.name delivery.provider.icon.prefix  \
0                         NaN                           NaN   
1                         NaN                           NaN   
2                         NaN                           NaN   
3                         NaN                           NaN   
4                         NaN                           NaN   

  delivery.provider.icon.sizes delivery.provider.name delivery.url  hasPerk  \
0                          NaN                    NaN          NaN    False   
1                          NaN                    NaN 

<h3>Clean up our dataframes and show head of restaurant dataframe</h3>

In [7]:
# keep only columns that include venue name, and anything that is associated with location
restaurant_filtered_columns = ['name', 'categories'] + [col for col in restaurant_dataframe.columns if col.startswith('location.')] + ['id']
restaurant_dataframe_filtered = restaurant_dataframe.loc[:, restaurant_filtered_columns]

hotel_filtered_columns = ['name', 'categories'] + [col2 for col2 in hotel_dataframe.columns if col2.startswith('location.')] + ['id']
hotel_dataframe_filtered = hotel_dataframe.loc[:, hotel_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
restaurant_dataframe_filtered['categories'] = restaurant_dataframe_filtered.apply(get_category_type, axis=1)
hotel_dataframe_filtered['categories'] = hotel_dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
restaurant_dataframe_filtered.columns = [column.split('.')[-1] for column in restaurant_dataframe_filtered.columns]
hotel_dataframe_filtered.columns = [column.split('.')[-1] for column in hotel_dataframe_filtered.columns]

restaurant_dataframe_filtered.head()
#hotel_dataframe_filtered.head()


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Oro Restaurant and Lounge,Tapas Restaurant,18 E Martin St,US,Raleigh,United States,at S Wilmington St,339,"[18 E Martin St (at S Wilmington St), Raleigh,...","[{'label': 'display', 'lat': 35.77698049500274...",35.776980,-78.638419,NaN,27601,NC,4f453e05e4b01cb12cdca861
1,Goodnight's Comedy Club & Restaurant,Comedy Club,861 W Morgan St,US,Raleigh,United States,at Tryon Hill Rd,1320,"[861 W Morgan St (at Tryon Hill Rd), Raleigh, ...","[{'label': 'display', 'lat': 35.78071237235952...",35.780712,-78.656106,NaN,27603,NC,4ad4c009f964a52017ed20e3
2,NC Seafood Restaurant,Seafood Restaurant,1201 Agriculture St,US,Raleigh,United States,at the Raleigh Farmers Market,2394,[1201 Agriculture St (at the Raleigh Farmers M...,"[{'label': 'display', 'lat': 35.76353992006933...",35.763540,-78.662750,NaN,27603,NC,4b74344ff964a520efcc2de3
3,State Farmers Market Restaurant,Southern / Soul Food Restaurant,1240 Farmers Market Dr,US,Raleigh,United States,at Centennial Pkwy,2599,"[1240 Farmers Market Dr (at Centennial Pkwy), ...","[{'label': 'display', 'lat': 35.76200847334365...",35.762008,-78.664142,NaN,27603,NC,4b084535f964a520d30723e3
4,Bida Manda Laotian Restaurant and Bar,Asian Restaurant,222 S Blount St,US,Raleigh,United States,NaN,484,"[222 S Blount St, Raleigh, NC 27601, United St...","[{'label': 'display', 'lat': 35.77722725876699...",35.777227,-78.636812,"Downtown Raleigh, Raleigh, NC",27601,NC,4ee404309911944900b07ad5


In [8]:
# The code was removed by Watson Studio for sharing.

<h3>Create map using Folium library - add choropleth layer and markers for restaurants/hotels</h3>

In [9]:
# generate map centred around the Raleigh, NC
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

def __iter__(self): return 0


# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about your possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_c7c0d08079c64a078a2ae20dede9ce26.get_object(Bucket='courseracapstone-donotdelete-pr-a2eccprmbo1m9n', Key='shown_geography.geojson')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 

raleigh_geojson = streaming_body_1.read()

body = client_c7c0d08079c64a078a2ae20dede9ce26.get_object(Bucket='courseracapstone-donotdelete-pr-a2eccprmbo1m9n',Key='shown_tract_kfr_rP_gP_pall.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

tracts = pd.read_csv(body)

# create choropleth map with income data from https://www.opportunityatlas.org/
venues_map.choropleth(
 geo_data=raleigh_geojson.decode(),
 data=tracts,
 name='choropleth',
 columns=['Tract', 'Household_Income'],
 key_on='feature.properties.id2',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Income Brackets'
)

folium.LayerControl().add_to(venues_map)

# add a red circle marker to represent the Raleigh city center
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Raleigh Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the restaurants as blue circle markers
for lat, lng, label in zip(restaurant_dataframe_filtered.lat, restaurant_dataframe_filtered.lng, restaurant_dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=str(label),
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# add the hotels as red circle markers
for lat2, lng2, label2 in zip(hotel_dataframe_filtered.lat, hotel_dataframe_filtered.lng, hotel_dataframe_filtered.name):
    folium.CircleMarker(
        [lat2, lng2],
        radius=5,
        color='red',
        popup=label2,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

    
# display map
venues_map

/opt/conda/envs/Python36/lib/python3.6/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


<h3>Configure data frames for k-means analysis</h3>

In [21]:
# configure data frames for k-means analysis

### --- setup restaurant_simple dataframe --- ###
# define the dataframe columns
column_names = ['Category', 'Latitude', 'Longitude'] 
# instantiate the dataframe
restaurant_simple = pd.DataFrame(columns=column_names)
# add to new dataframe
for lat, lng, label in zip(restaurant_dataframe_filtered.lat, restaurant_dataframe_filtered.lng, restaurant_dataframe_filtered.categories):
    restaurant_simple = restaurant_simple.append({'Category': label,
                                          'Latitude': lat,
                                          'Longitude': lng}, ignore_index=True)
### ---    end restaurant_simple setup   --- ###

### --- setup hotel_simple dataframe --- ###
# define column names
column_names2 = ['Name', 'Latitude', 'Longitude'] 
# instantiate the dataframe
hotel_simple = pd.DataFrame(columns=column_names2)
# add new to dataframe
for lat2, lng2, label2 in zip(hotel_dataframe_filtered.lat, hotel_dataframe_filtered.lng, hotel_dataframe_filtered.name):
    hotel_simple = hotel_simple.append({'Name': label2,
                                       'Latitude' : lat2,
                                       'Longitude': lng2}, ignore_index=True)
### ---  end hotel_simple setup  --- ###
    

<h3>Perform k-means analysis</h3>

In [22]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# now perform clustering
# set number of clusters
krestclusters = 5
khotelclusters = 3

### --- create restaurant clusters --- ###
restaurant_simple_cluster = restaurant_simple.drop('Category', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=krestclusters, random_state=0).fit(restaurant_simple_cluster)
# add clustering labels
restaurant_simple_cluster.insert(0, 'Cluster Labels', kmeans.labels_)
### --- end restaurant cluster create --- ###

### ---   create hotel clusters   --- ###
hotel_simple_cluster = hotel_simple.drop('Name', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=khotelclusters, random_state=0).fit(hotel_simple_cluster)
# add clustering labels
hotel_simple_cluster.insert(0, 'Cluster Labels', kmeans.labels_)
### ---   end restaurant cluster create   --- ###


<h3>Create final choropleth map with k-means data groupings</h3>

In [19]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# plot on new map
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the restaurant clusters
x = np.arange(krestclusters)
ys = [i + x + (i*x)**2 for i in range(krestclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# set color scheme for hotel clusters
grey_array = ['#D3D3D3','#A9A9A9','#606060']

# create choropleth map with income data from https://www.opportunityatlas.org/
map_clusters.choropleth(
 geo_data=raleigh_geojson.decode(),
 data=tracts,
 name='choropleth',
 columns=['Tract', 'Household_Income'],
 key_on='feature.properties.id2',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Income Brackets'
)


# add restaurant markers to the map
for lat, lon, cluster in zip(restaurant_simple_cluster['Latitude'], restaurant_simple_cluster['Longitude'], restaurant_simple_cluster['Cluster Labels']):
    label = folium.Popup('Restaurant cluster: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       

# add restaurant markers to the map
for lat, lon, cluster in zip(hotel_simple_cluster['Latitude'], hotel_simple_cluster['Longitude'], hotel_simple_cluster['Cluster Labels']):
    label = folium.Popup('Hotel cluster: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=grey_array[cluster-1],
        fill=True,
        fill_color=grey_array[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
        

# add layer control so user can toggle choropleth
folium.LayerControl().add_to(map_clusters)


# show the map
map_clusters

/opt/conda/envs/Python36/lib/python3.6/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
